# Make the final catalogue

We want to take all the columns. We will save a catalogue for each patch which can be stacked to produce the final catalogues for serving publicly. The is_inner flags are used to remove the overlap regions. As far as possible we try to mimic the tables produced for HSC PDR2.

This is a test which will then be run at scale for the large area survey runs.

In [6]:
import json
import numpy as np
import lsst.daf.persistence as dafPersist

from astropy.table import Table

In [7]:
hscBands = ['G', 'R', 'I', 'Z', 'Y']
vistaBands = ['Z', 'Y', 'J', 'H', 'Ks']


In [8]:
BUTLER_LOC = '../../dmu4/dmu4_Example/data_g2'
butler =  dafPersist.Butler(inputs='{}/rerun/coadd'.format(BUTLER_LOC))

ConstructorError: while constructing a Python object
cannot find module 'lsst.obs.vista.vistaMapper' (No module named 'lsst.obs.vista.vistaMapper')
  in "/Users/raphaelshirley/Documents/github/lsst-ir-fusion/dmu4/dmu4_Example/data_g2/rerun/coadd/repositoryCfg.yaml", line 2, column 10

In [1]:
!mkdir data

## 0. Make a csv file of columns

In [5]:
ksSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-Ks', 'tract': 8523, 'patch': '3,4'})

NameError: name 'butler' is not defined

## 1. Investigate Butler and check joining on ids

In [12]:
ySources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-Y', 'tract': tract, 'patch': patch})
ksSources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-Ks', 'tract': tract, 'patch': patch})

DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/VISTA-Y/8524/5,5/forcedSrc-VISTA-Y-8524-5,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/VISTA-Ks/8524/5,5/forcedSrc-VISTA-Ks-8524-5,5.fits


In [13]:
ySources.asAstropy()[:5]

id,coord_ra,coord_dec,parent,deblend_nChild,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,base_SdssCentroid_flag,base_SdssCentroid_flag_edge,base_SdssCentroid_flag_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_SdssCentroid_flag_resetToPeak,base_SdssCentroid_flag_badError,base_TransformedCentroid_x,slot_Centroid_x,base_TransformedCentroid_y,slot_Centroid_y,base_TransformedCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_GaussianFlux_flag_badCentroid,base_InputCount_flag_badCentroid,base_LocalBackground_flag_badCentroid,base_PsfFlux_flag_badCentroid,base_SdssCentroid_flag_badInitialCentroid,base_SdssShape_flag_badCentroid,base_Variance_flag_badCentroid,slot_Centroid_flag,base_InputCount_flag,base_InputCount_value,base_InputCount_flag_noInputs,base_SdssShape_xx,base_SdssShape_yy,base_SdssShape_xy,base_SdssShape_xxErr,base_SdssShape_yyErr,base_SdssShape_xyErr,base_SdssShape_x,base_SdssShape_y,base_SdssShape_instFlux,base_SdssShape_instFluxErr,base_SdssShape_psf_xx,slot_PsfShape_xx,base_SdssShape_psf_yy,slot_PsfShape_yy,base_SdssShape_psf_xy,slot_PsfShape_xy,base_SdssShape_instFlux_xx_Cov,base_SdssShape_instFlux_yy_Cov,base_SdssShape_instFlux_xy_Cov,base_SdssShape_flag,base_SdssShape_flag_unweightedBad,base_SdssShape_flag_unweighted,base_SdssShape_flag_shift,base_SdssShape_flag_maxIter,base_SdssShape_flag_psf,base_TransformedShape_xx,slot_Shape_xx,base_TransformedShape_yy,slot_Shape_yy,base_TransformedShape_xy,slot_Shape_xy,base_TransformedShape_flag,base_GaussianFlux_flag_badShape,slot_Shape_flag,base_CircularApertureFlux_3_0_instFlux,base_CircularApertureFlux_3_0_instFluxErr,base_CircularApertureFlux_3_0_flag,base_CircularApertureFlux_3_0_flag_apertureTruncated,base_CircularApertureFlux_3_0_flag_sincCoeffsTruncated,base_CircularApertureFlux_4_5_instFlux,base_CircularApertureFlux_4_5_instFluxErr,base_CircularApertureFlux_4_5_flag,base_CircularApertureFlux_4_5_flag_apertureTruncated,base_CircularApertureFlux_4_5_flag_sincCoeffsTruncated,base_CircularApertureFlux_6_0_instFlux,base_CircularApertureFlux_6_0_instFluxErr,base_CircularApertureFlux_6_0_flag,base_CircularApertureFlux_6_0_flag_apertureTruncated,base_CircularApertureFlux_6_0_flag_sincCoeffsTruncated,base_CircularApertureFlux_9_0_instFlux,base_CircularApertureFlux_9_0_instFluxErr,base_CircularApertureFlux_9_0_flag,base_CircularApertureFlux_9_0_flag_apertureTruncated,base_CircularApertureFlux_9_0_flag_sincCoeffsTruncated,base_CircularApertureFlux_12_0_instFlux,base_CircularApertureFlux_12_0_instFluxErr,base_CircularApertureFlux_12_0_flag,base_CircularApertureFlux_12_0_flag_apertureTruncated,base_CircularApertureFlux_17_0_instFlux,base_CircularApertureFlux_17_0_instFluxErr,base_CircularApertureFlux_17_0_flag,base_CircularApertureFlux_17_0_flag_apertureTruncated,base_CircularApertureFlux_25_0_instFlux,base_CircularApertureFlux_25_0_instFluxErr,base_CircularApertureFlux_25_0_flag,base_CircularApertureFlux_25_0_flag_apertureTruncated,base_CircularApertureFlux_35_0_instFlux,base_CircularApertureFlux_35_0_instFluxErr,base_CircularApertureFlux_35_0_flag,base_CircularApertureFlux_35_0_flag_apertureTruncated,base_CircularApertureFlux_50_0_instFlux,base_CircularApertureFlux_50_0_instFluxErr,base_CircularApertureFlux_50_0_flag,base_CircularApertureFlux_50_0_flag_apertureTruncated,base_CircularApertureFlux_70_0_instFlux,base_CircularApertureFlux_70_0_instFluxErr,base_CircularApertureFlux_70_0_flag,base_CircularApertureFlux_70_0_flag_apertureTruncated,base_GaussianFlux_instFlux,base_GaussianFlux_instFluxErr,base_GaussianFlux_flag,base_LocalBackground_instFlux,base_LocalBackground_instFluxErr,base_LocalBackground_flag,base_LocalBackground_flag_noGoodPixels,base_LocalBackground_flag_noPsf,base_PixelFlags_flag,base_PixelFlags_flag_offimage,base_PixelFlags_flag_edge,base_PixelFlags_flag_interpolated,base_PixelFlags_flag_saturated,base_PixelFlags_flag_cr,base_PixelFlags_flag_bad,ba

In [14]:
ksSources.asAstropy()[:5]

id,coord_ra,coord_dec,parent,deblend_nChild,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,base_SdssCentroid_flag,base_SdssCentroid_flag_edge,base_SdssCentroid_flag_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_SdssCentroid_flag_resetToPeak,base_SdssCentroid_flag_badError,base_TransformedCentroid_x,slot_Centroid_x,base_TransformedCentroid_y,slot_Centroid_y,base_TransformedCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_GaussianFlux_flag_badCentroid,base_InputCount_flag_badCentroid,base_LocalBackground_flag_badCentroid,base_PsfFlux_flag_badCentroid,base_SdssCentroid_flag_badInitialCentroid,base_SdssShape_flag_badCentroid,base_Variance_flag_badCentroid,slot_Centroid_flag,base_InputCount_flag,base_InputCount_value,base_InputCount_flag_noInputs,base_SdssShape_xx,base_SdssShape_yy,base_SdssShape_xy,base_SdssShape_xxErr,base_SdssShape_yyErr,base_SdssShape_xyErr,base_SdssShape_x,base_SdssShape_y,base_SdssShape_instFlux,base_SdssShape_instFluxErr,base_SdssShape_psf_xx,slot_PsfShape_xx,base_SdssShape_psf_yy,slot_PsfShape_yy,base_SdssShape_psf_xy,slot_PsfShape_xy,base_SdssShape_instFlux_xx_Cov,base_SdssShape_instFlux_yy_Cov,base_SdssShape_instFlux_xy_Cov,base_SdssShape_flag,base_SdssShape_flag_unweightedBad,base_SdssShape_flag_unweighted,base_SdssShape_flag_shift,base_SdssShape_flag_maxIter,base_SdssShape_flag_psf,base_TransformedShape_xx,slot_Shape_xx,base_TransformedShape_yy,slot_Shape_yy,base_TransformedShape_xy,slot_Shape_xy,base_TransformedShape_flag,base_GaussianFlux_flag_badShape,slot_Shape_flag,base_CircularApertureFlux_3_0_instFlux,base_CircularApertureFlux_3_0_instFluxErr,base_CircularApertureFlux_3_0_flag,base_CircularApertureFlux_3_0_flag_apertureTruncated,base_CircularApertureFlux_3_0_flag_sincCoeffsTruncated,base_CircularApertureFlux_4_5_instFlux,base_CircularApertureFlux_4_5_instFluxErr,base_CircularApertureFlux_4_5_flag,base_CircularApertureFlux_4_5_flag_apertureTruncated,base_CircularApertureFlux_4_5_flag_sincCoeffsTruncated,base_CircularApertureFlux_6_0_instFlux,base_CircularApertureFlux_6_0_instFluxErr,base_CircularApertureFlux_6_0_flag,base_CircularApertureFlux_6_0_flag_apertureTruncated,base_CircularApertureFlux_6_0_flag_sincCoeffsTruncated,base_CircularApertureFlux_9_0_instFlux,base_CircularApertureFlux_9_0_instFluxErr,base_CircularApertureFlux_9_0_flag,base_CircularApertureFlux_9_0_flag_apertureTruncated,base_CircularApertureFlux_9_0_flag_sincCoeffsTruncated,base_CircularApertureFlux_12_0_instFlux,base_CircularApertureFlux_12_0_instFluxErr,base_CircularApertureFlux_12_0_flag,base_CircularApertureFlux_12_0_flag_apertureTruncated,base_CircularApertureFlux_17_0_instFlux,base_CircularApertureFlux_17_0_instFluxErr,base_CircularApertureFlux_17_0_flag,base_CircularApertureFlux_17_0_flag_apertureTruncated,base_CircularApertureFlux_25_0_instFlux,base_CircularApertureFlux_25_0_instFluxErr,base_CircularApertureFlux_25_0_flag,base_CircularApertureFlux_25_0_flag_apertureTruncated,base_CircularApertureFlux_35_0_instFlux,base_CircularApertureFlux_35_0_instFluxErr,base_CircularApertureFlux_35_0_flag,base_CircularApertureFlux_35_0_flag_apertureTruncated,base_CircularApertureFlux_50_0_instFlux,base_CircularApertureFlux_50_0_instFluxErr,base_CircularApertureFlux_50_0_flag,base_CircularApertureFlux_50_0_flag_apertureTruncated,base_CircularApertureFlux_70_0_instFlux,base_CircularApertureFlux_70_0_instFluxErr,base_CircularApertureFlux_70_0_flag,base_CircularApertureFlux_70_0_flag_apertureTruncated,base_GaussianFlux_instFlux,base_GaussianFlux_instFluxErr,base_GaussianFlux_flag,base_LocalBackground_instFlux,base_LocalBackground_instFluxErr,base_LocalBackground_flag,base_LocalBackground_flag_noGoodPixels,base_LocalBackground_flag_noPsf,base_PixelFlags_flag,base_PixelFlags_flag_offimage,base_PixelFlags_flag_edge,base_PixelFlags_flag_interpolated,base_PixelFlags_flag_saturated,base_PixelFlags_flag_cr,base_PixelFlags_flag_bad,ba

In [15]:
len(ySources),len(ksSources)

(36204, 36204)

In [7]:
for patch in np.nditer(np.meshgrid(np.arange(3,6),np.arange(3,6))):
    tract = 8524
    patch = '{},{}'.format(patch[0],patch[1])
    print(patch)
   

3,3
4,3
5,3
3,4
4,4
5,4
3,5
4,5
5,5


In [ ]:
full_cat = Table()
for f in y_stack_files[6:7]:
    tract = 8524
    patch = str(f[-8:-5])
    print(patch)
    #zSources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-Z', 'tract': tract, 'patch': patch})
    ySources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-Y', 'tract': tract, 'patch': patch})
    jSources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-J', 'tract': tract, 'patch': patch})
    hSources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-H', 'tract': tract, 'patch': patch})
    ksSources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-Ks', 'tract': tract, 'patch': patch})
    #yCoaddPhotoCalib = butler.get('deepCoadd_calexp_photoCalib',  {'filter': 'VISTA-Y', 'tract': tract, 'patch': patch})
    #jCoaddPhotoCalib = butler.get('deepCoadd_calexp_photoCalib',  {'filter': 'VISTA-J', 'tract': tract, 'patch': patch})
    #hCoaddPhotoCalib = butler.get('deepCoadd_calexp_photoCalib',  {'filter': 'VISTA-H', 'tract': tract, 'patch': patch})
    #ksCoaddPhotoCalib = butler.get('deepCoadd_calexp_photoCalib',  {'filter': 'VISTA-Ks', 'tract': tract, 'patch': patch})
    #zCoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': 'VISTA-Z', 'tract': tract, 'patch': patch})
    #zCoaddPhotoCalib = zCoaddCalexp.getPhotoCalib()
    yCoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': 'VISTA-Y', 'tract': tract, 'patch': patch})
    yCoaddPhotoCalib = yCoaddCalexp.getPhotoCalib()
    jCoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': 'VISTA-J', 'tract': tract, 'patch': patch})
    jCoaddPhotoCalib = jCoaddCalexp.getPhotoCalib()
    hCoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': 'VISTA-H', 'tract': tract, 'patch': patch})
    hCoaddPhotoCalib = hCoaddCalexp.getPhotoCalib()
    ksCoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': 'VISTA-Ks', 'tract': tract, 'patch': patch})
    ksCoaddPhotoCalib = ksCoaddCalexp.getPhotoCalib()
    f_type = 'base_PsfFlux' #'base_PsfFlux' 'base_CircularApertureFlux_3_0'
    yMags = yCoaddPhotoCalib.instFluxToMagnitude(ySources, f_type)
    jMags = jCoaddPhotoCalib.instFluxToMagnitude(jSources, f_type)
    hMags = hCoaddPhotoCalib.instFluxToMagnitude(hSources, f_type)
    ksMags = ksCoaddPhotoCalib.instFluxToMagnitude(ksSources, f_type)
    
    yFlux = yCoaddPhotoCalib.instFluxToNanojansky(ySources, f_type)
    jFlux = jCoaddPhotoCalib.instFluxToNanojansky(jSources, f_type)
    hFlux = hCoaddPhotoCalib.instFluxToNanojansky(hSources, f_type)
    ksFlux = ksCoaddPhotoCalib.instFluxToNanojansky(ksSources, f_type)
    
    #zMeasSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-Z', 'tract': tract, 'patch': patch})
    yMeasSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-Y', 'tract': tract, 'patch': patch})
    jMeasSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-J', 'tract': tract, 'patch': patch})
    hMeasSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-H', 'tract': tract, 'patch': patch})
    ksMeasSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-Ks', 'tract': tract, 'patch': patch})

    fMeas_type = 'base_CircularApertureFlux_12_0' #'base_PsfFlux' 'base_CircularApertureFlux_2_0'
    #zMeasMags = zCoaddPhotoCalib.instFluxToMagnitude(zMeasSources, fMeas_type)
    yMeasMags = yCoaddPhotoCalib.instFluxToMagnitude(yMeasSources, fMeas_type)
    jMeasMags = jCoaddPhotoCalib.instFluxToMagnitude(jMeasSources, fMeas_type)
    hMeasMags = hCoaddPhotoCalib.instFluxToMagnitude(hMeasSources, fMeas_type)
    ksMeasMags = ksCoaddPhotoCalib.instFluxToMagnitude(ksMeasSources, fMeas_type)
    
    yMeasFlux = yCoaddPhotoCalib.instFluxToNanojansky(yMeasSources, fMeas_type)
    ksMeasFlux = ksCoaddPhotoCalib.instFluxToNanojansky(ksMeasSources, fMeas_type)

    #isDeblended = ySources['deblend_nChild'] == 0
    
    
    
    
    sn5 = (yMeasFlux[:,0]/yMeasFlux[:,1] > 5) & (ksMeasFlux[:,0]/ksMeasFlux[:,1] > 5)
    isDeblended = ksSources['deblend_nChild'] == 0
    refTable = butler.get('deepCoadd_ref', {'filter': 'VISTA-Y^VISTA-Ks', 'tract': tract, 'patch': patch})
    #refTable
    inInnerRegions = refTable['detect_isPatchInner'] & refTable['detect_isTractInner']
    isSkyObject = refTable['merge_peak_sky']
    isPrimary = refTable['detect_isPrimary']
    isStellar = refTable['base_ClassificationExtendedness_value'] < 1.
    ksSources.getSchema().extract('{}_*'.format(f_type))
    isGoodFlux = ~ksSources['{}_flag'.format(f_type)]
    selected = isPrimary & isGoodFlux & inInnerRegions & isStellar  
    cat = Table()

    #cat['m_vista_z'] = zMags[:,0]
    #cat['merr_vista_z'] = zMags[:,1]
    #cat['m_ap_vista_z'] = zMeasMags[:,0]
    #cat['merr_ap_vista_z'] = zMeasMags[:,1]    
                                                     
    cat['m_vista_y'] = yMags[:,0]
    cat['merr_vista_y'] = yMags[:,1]
    cat['f_vista_y'] = yFlux[:,0]
    cat['ferr_vista_y'] = yFlux[:,1]
    cat['m_ap_vista_y'] = yMeasMags[:,0]
    cat['merr_ap_vista_y'] = yMeasMags[:,1]
    cat['f_ap_vista_y'] = yMeasFlux[:,0]
    cat['ferr_ap_vista_y'] = yMeasFlux[:,1]
                
    cat['m_vista_j'] = jMags[:,0]
    cat['merr_vista_j'] = jMags[:,1]
    cat['m_ap_vista_j'] = jMeasMags[:,0]
    cat['merr_ap_vista_j'] = jMeasMags[:,1]

                                                     
    cat['m_vista_h'] = hMags[:,0]
    cat['merr_vista_h'] = hMags[:,1]
    cat['m_ap_vista_h'] = hMeasMags[:,0]
    cat['merr_ap_vista_h'] = hMeasMags[:,1]
                                                     
    cat['m_vista_ks'] = ksMags[:,0]
    cat['merr_vista_ks'] = ksMags[:,1]
    cat['m_ap_vista_ks'] = ksMeasMags[:,0]
    cat['merr_ap_vista_ks'] = ksMeasMags[:,1]
    cat['f_ap_vista_ks'] = ksMeasFlux[:,0]
    cat['ferr_ap_vista_ks'] = ksMeasFlux[:,1]

    cat['ra'] = ksSources['coord_ra']
    #cat['ra'] =ksMeasSources['coord_ra']
    cat['dec'] = ksSources['coord_dec']
    #cat['dec'] = ksMeasSources['coord_dec']
    cat['flag'] = selected
    full_cat = vstack([full_cat,cat])